# Project 3: Understanding User Behavior
**Project Team: Jude Wentian Zhu, Rohit Barkshi, Rathin Bector**

## Description of Project Files


<span style="color:red">**Need to fill in**</span>

## Project Goal

<span style="color:red">**Need to fill in**</span>

## Summary of Data Pipeline

<span style="color:red">**Need to fill in**</span>

## Data Pipeline Steps

### Step 1: Spin up Docker-Compose and Link Pypark to Jupyter Notebook

1. Spin up the cluster

In [1]:
!docker-compose up -d

Creating network "project_3_default" with the default driver
Creating project_3_presto_1 ... 
Creating project_3_redis_1  ... 
Creating project_3_cloudera_1 ... 
Creating project_3_zookeeper_1 ... 
Creating project_3_mids_1      ... 
ting project_3_cloudera_1  ... doneCreating project_3_spark_1     ... 
Creating project_3_kafka_1     ... mdone
ting project_3_kafka_1     ... done

2. Exec a bash shell in the spark container on teminal.
```console
docker-compose exec spark bash
```


3. Create a symbolic link from the spark directory to /w205 :
```console
ln -s /w205 w205
```


4. Exit the container
```console
exit
```


5. Check out Hadoop

In [2]:
!docker-compose exec cloudera hadoop fs -ls /tmp/

Found 3 items
drwxrwxrwt   - mapred mapred              0 2016-04-06 02:26 /tmp/hadoop-yarn
drwx-wx-wx   - hive   supergroup          0 2021-08-05 04:36 /tmp/hive
drwxrwxrwt   - mapred hadoop              0 2016-04-06 02:28 /tmp/logs


### Step 2: Launch Kafka and Flask

1. Create a kafka topic called events

In [3]:
!docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

Created topic events.


2. Install dependencies for flask app on mids container

In [4]:
!docker-compose exec mids pip install redis
!docker-compose exec mids pip install numpy==1.14.6

The directory '/w205/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/w205/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
    100% |################################| 81kB 6.1MB/s eta 0:00:01
You are using pip version 8.1.1, however version 21.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
The directory '/w205/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/w205/.cache/pip' or its parent directory is not 

3. Run flask app on teminal
```console
docker-compose exec mids env FLASK_APP=/w205/project_3/game_api.py flask run --host 0.0.0.0
```


4. Read events from kafka

In [1]:
!docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning -e

% Reached end of topic events [0] at offset 0: exiting


### Step 3: Use Spark to Batch Analysis

In [3]:
# Import Packages
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, from_json
from pyspark.sql.types import StructType, StructField, StringType
import re

In [6]:
@udf('boolean')
def is_purchase_weapon(event_as_json):
    event = json.loads(event_as_json)
    # m = re.match('purchase',event['event_type'])
    if event['event_type'] == 'purchase_weapon':
        return True
    return False

In [7]:
@udf('boolean')
def is_purchase_shield(event_as_json):
    event = json.loads(event_as_json)
    # m = re.match('purchase',event['event_type'])
    if event['event_type'] == 'purchase_shield':
        return True
    return False

In [ ]:
def purchase_sword_event_schema():
    """
    root
    |-- Accept: string (nullable = true)
    |-- Host: string (nullable = true)
    |-- User-Agent: string (nullable = true)
    |-- event_type: string (nullable = true)
    |-- timestamp: string (nullable = true)
    """
    return StructType([
        StructField("Accept", StringType(), True),
        StructField("Host", StringType(), True),
        StructField("User-Agent", StringType(), True),
        StructField("event_type", StringType(), True),
    ])

In [ ]:
spark = SparkSession \
    .builder \
    .appName("ExtractEventsJob") \
    .getOrCreate()

In [8]:
raw_events = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "events") \
    .load()

In [5]:
purchase_weapon = raw_events \
    .filter(is_purchase_weapon(raw_events.value.cast('string'))) \
    .select(raw_events.value.cast('string').alias('raw_event'),
            raw_events.timestamp.cast('string'),
            from_json(raw_events.value.cast('string'),
                      purchase_sword_event_schema()).alias('json')) \
    .select('raw_event', 'timestamp', 'json.*')

<bound method DataFrame.count of DataFrame[raw: string, timestamp: string]>

### Step 4: Use 